## IRCNN model implementation in Keras

In [2]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, Activation, Input, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
import os
import numpy as np
%matplotlib inline

Next, we need to build the model. The architecture, as given in [the paper](http://www4.comp.polyu.edu.hk/%7Ecslzhang/paper/IRCNN_CVPR17.pdf), has the below architecture:
![IRCNN Architecture](./architecture.png)
The paper uses (3 x 3) dilated convolutions in each layer. Instead of the proposed loss, this implementation uses the MSE loss function.

In [3]:
def IRCNN(input_shape=(None, None, 3)):
    X_input = Input(input_shape)
    
    X = Conv2D(filters=1, kernel_size=3, dilation_rate=1)(X_input)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=1, kernel_size=3, dilation_rate=2)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=1, kernel_size=3, dilation_rate=3)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=1, kernel_size=3, dilation_rate=4)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=1, kernel_size=3, dilation_rate=3)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=1, kernel_size=3, dilation_rate=2)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=1, kernel_size=3, dilation_rate=1)(X)
    
    model = Model(inputs=X_input, outputs=X, name='IRCNN')
    return model

In [5]:
model = IRCNN(input_shape=(None, None, 1))

In [6]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [7]:
files = os.listdir('VOC2012/JPEGImages')
n_files = len(files)

train = files[:int(0.70 * n_files)]
test = files[int(0.70 * n_files):]
print(len(train), len(test))

11987 5138


In [12]:
X_train_files = ['VOC2012/rayleigh/' + x.split('.')[0] + '.png' for x in train]
Y_train_files = ['VOC2012/red_channel/' + x.split('.')[0] + '.png' for x in train]

X_test_files = ['VOC2012/rayleigh/' + x.split('.')[0] + '.png' for x in test]
Y_test_files = ['VOC2012/red_channel/' + x.split('.')[0] + '.png' for x in test]

In [13]:
epochs = 2

In [14]:
training_data = list(zip(X_train_files, Y_train_files))

In [34]:
for e in range(epochs):
    for i in range(0, len(training_data), 32):
        batch = training_data[i:i+32]
        X_train = list(list(zip(*batch))[0])
        Y_train = list(zip(*batch))[1]
        
        X = []
        Y = []
        for x in X_train:
            img = plt.imread(x)
            img = img.reshape(*(img.shape), 1)
            X.append(img)
        
        for y in Y_train:
            img = plt.imread(y)
            img = img.reshape(*(img.shape), 1)
            Y.append(img)
        
        model.train_on_batch(np.array(X), np.array(Y))

ValueError: Error when checking input: expected input_2 to have 4 dimensions, but got array with shape (32, 1)